# S4. Fine-tuning de modelo pre-entrenado para la clasificación de CIFAR-10
En esta sesión veremos como realizar el fine-tuning (ajuste fino) de una arquitectura de red [EfficientNet](https://arxiv.org/abs/1905.11946), más concretamente veremos su [B0 disponible en Keras](https://keras.io/api/applications/efficientnet/#efficientnetb0-function) pre-entrenada con la base de datos de imágenes [ImageNet](https://www.image-net.org) para la clasificación de CIFAR-10. 

### Carga de datos
La utilización de una red pre-entrenada conlleva preprocesar nuestros datos con el mismo preproceso que se empleó para los datos de entrenamiento de la red pre-entrenada. Si realizamos este preproceso para nuestro conjunto de entrenamiento, esto requeriría aproximadamente 40GB, por lo que es conveniente realizar este preproceso bajo demanda. Es decir, aplicaremos el preproceso para cada batch en el momento que vaya a ser utilizado.      

La aplicación de este preproceso bajo demanda está ya implementado en el módulo [tensorflow_datasets](https://www.tensorflow.org/datasets), así que en esta sesión cargaremos y manipularemos CIFAR-10 como un objeto [Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) utilizando este módulo.

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
train_data, test_data = tfds.load('cifar10', split=['train', 'test'], as_supervised=True)
train_size = len(train_data)

### Preproceso

Primero definimos una función que aplica el preproceso necesario a una muestra de entrenamiento (imagen, etiqueta de clase). Esto incluye redimensionar la imagen a 299 x 299, el preproceso específico de la red InceptionV3 y convertir la etiqueta de clase a one-hot encoding.

In [3]:
from keras.applications.efficientnet import preprocess_input

img_size = (299, 299)
num_classes = 10

def preprocess(image, label):
    image = tf.image.resize(image, img_size)
    image = tf.cast(image, tf.float32)
    image = preprocess_input(image)
    label = tf.one_hot(label, num_classes)
    return image, label

A continuación indicamos que la función anteriormente definida se aplicará a cada imagen cuando sea necesario. Para ello utilizamos la función [map()](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map) del objeto [Dataset](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) y le pasamos como parámetro la función de preproceso que queremos que sea aplique a cada muestra del conjunto de datos.

In [4]:
train_data = train_data.map(preprocess)
test_data = test_data.map(preprocess)

Las funciones [take()](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#take) y [skip()](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#skip) combinadas permiten definir los conjuntos de entrenamiento y validación como nuevos Datasets. 

In [5]:
train_size = int(0.8 * train_size)
train_dataset = train_data.take(train_size)
val_dataset = train_data.skip(train_size)
test_dataset = test_data

print(len(train_dataset),len(val_dataset))

40000 10000


### Carga del modelo pre-entrenado

Seguidamente procedemos con la carga del modelo EfficientNet B0 con los pesos resultantes de entrenarlo con la base de datos Imagenet, pero no queremos que el modelo incluya la capa de salida (include_top=False) que por defecto es una softmax de 1000 clases.

In [10]:
from keras.applications.efficientnet import EfficientNetB0

model = EfficientNetB0(input_shape=img_size + (3,),include_top=False, weights='imagenet')

### Preparación del modelo pre-entrenado

Vamos a preparar la red EfficientNet B0 para ser entrenada (fine-tuning) con CIFAR-10. Dado el número de parámetros de este modelo (21M), nos limitaremos a utilizarlo con los valores por defecto y añadiremos una capa GlobalAveragePooling + MLP seguida de una softmax de 10 neuronas (10 clases) acorde a CIFAR-10 que sí que entrenaremos.

In [11]:
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.models import Model

for layer in model.layers:
    layer.trainable = False

x = GlobalAveragePooling2D()(model.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=model.input, outputs=output)

Compilamos el modelo con los mismos parámetros que en sesiones anteriores.

In [12]:
from keras.optimizers import Adam

opt=Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

Entrenamos el modelo utilizando los conjuntos de datos organizado en batches y que son cargados en memoria dinámicamente.

In [13]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.models import load_model

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

epochs=10
batch_size=32
train_dataset_batched = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset_batched = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
history = model.fit(train_dataset_batched,
                    epochs=epochs,
                    verbose=1,
                    validation_data=val_dataset_batched,
                    callbacks=[reduce_lr,checkpoint])

Epoch 1/10


2023-12-20 18:00:20.405353: W external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:225] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.9
2023-12-20 18:00:20.405370: W external/local_xla/xla/stream_executor/gpu/asm_compiler.cc:228] Used ptxas at ptxas
2023-12-20 18:00:20.405404: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2023-12-20 18:00:20.496199: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2023-12-20 18:00:20.496271: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] Failed to compile generated PTX with ptxas. Falling back to compilation by driver.
2023-12-20 18:00:20.498464: W tensorflow/compiler/mlir/tools/kernel_gen/transforms/gpu_kernel_to_blob_pass.cc:191] 

1250/1250 [==============================] - ETA: 0s - loss: 0.5379 - accuracy: 0.8129

2023-12-20 18:02:11.731385: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.



Epoch 1: val_accuracy improved from -inf to 0.88640, saving model to best_model.h5


/home/jorcisai/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1250/1250 [==============================] - 144s 112ms/step - loss: 0.5379 - accuracy: 0.8129 - val_loss: 0.3308 - val_accuracy: 0.8864 - lr: 0.0010
Epoch 2/10
1250/1250 [==============================] - ETA: 0s - loss: 0.4242 - accuracy: 0.8523
Epoch 2: val_accuracy improved from 0.88640 to 0.89280, saving model to best_model.h5
1250/1250 [==============================] - 138s 110ms/step - loss: 0.4242 - accuracy: 0.8523 - val_loss: 0.3066 - val_accuracy: 0.8928 - lr: 0.0010
Epoch 3/10
1250/1250 [==============================] - ETA: 0s - loss: 0.3846 - accuracy: 0.8661
Epoch 3: val_accuracy improved from 0.89280 to 0.89940, saving model to best_model.h5
1250/1250 [==============================] - 138s 110ms/step - loss: 0.3846 - accuracy: 0.8661 - val_loss: 0.2962 - val_accuracy: 0.8994 - lr: 0.0010
Epoch 4/10
1250/1250 [==============================] - ETA: 0s - loss: 0.3582 - accuracy: 0.8742
Epoch 4: val_accuracy did not improve from 0.89940
1250/1250 [======================

### Cargar el mejor modelo y evaluarlo con el test set

In [14]:
model = load_model('best_model.h5')
test_dataset_batched = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
score = model.evaluate(test_dataset_batched, verbose=0)
print(f'Test loss: {score[0]*100:.2f}')
print(f'Test accuracy: {score[1]*100:.2f}')

Test loss: 26.79
Test accuracy: 91.35
